In [ ]:
!pip install pingouin

import pandas as pd
import pingouin as pg
import numpy as np

def calculate_icc(csv_file):
    data = pd.read_csv(csv_file)
    data_long = data.melt(var_name='raters', value_name='ratings', ignore_index=False)
    data_long.reset_index(inplace=True)
    data_long.rename(columns={'index': 'targets'}, inplace=True)
    icc_results = pg.intraclass_corr(data=data_long, targets='targets', raters='raters', ratings='ratings')
    return icc_results, data_long

def calculate_sems_and_mdcs(icc_value, data_long, suffix):
    s = np.std(data_long['ratings'], ddof=1)
    sem = s * np.sqrt(1 - icc_value)
    ratings_diff = data_long.groupby('targets')['ratings'].diff().dropna()
    sd_diff = np.std(ratings_diff, ddof=1)
    sem_diff = sd_diff / np.sqrt(2)
    
    mdc95 = sem * 1.96 * np.sqrt(2)
    mdc95_diff = sem_diff * 1.96 * np.sqrt(2)
    
    print(f"Standard Deviation (s): {s:.4f}")
    print(f"SEM{suffix} (from ICC): {sem:.4f}")
    print(f"SEM (using SD of mean differences for ICC): {sem_diff:.4f}")
    print(f"MDC95 (from ICC): {mdc95:.4f}")
    print(f"MDC95 (using SD of mean differences for ICC): {mdc95_diff:.4f}")

def main_icc(csv_file):
    try:
        icc_results, data_long = calculate_icc(csv_file)
    except FileNotFoundError:
        print(f"Error: File '{csv_file}' not found.")
        return
    except Exception as e:
        print(f"An error occurred: {e}")
        return

    for icc_type in ['ICC1', 'ICC2', 'ICC3']:
        icc_row = icc_results[icc_results['Type'] == icc_type]
        if not icc_row.empty:
            icc_value = icc_row['ICC'].values[0]
            icc_pval = icc_row['pval'].values[0]
            icc_ci95 = icc_row['CI95%'].values[0]

            print(f"\n{icc_type} Value: {icc_value:.4f}")
            print(f"{icc_type} p-value: {icc_pval:.2e}")
            print(f"{icc_type} 95% Confidence Interval (CI95%): {icc_ci95}")

            calculate_sems_and_mdcs(icc_value, data_long, icc_type[-1])
        else:
            print(f"{icc_type} is not found in the results.")

# 使用例
main_icc('deta_x.csv')

